<a href="https://colab.research.google.com/github/mindfulcoder49/NorthShoreAI/blob/main/Create_And_Deploy_A_Personal_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Create and Deploy a personal RAG app

Welcome to the 2nd Meetup of North Shore AI Developers. Last time we created a full stack app that ran completely inside of a Colab notebook with the help of Gradio. This time we are going to use Google Colab to create our app and deploy it to github, after which we will use Vercel to deploy it in a persistent way.

For this notebook, you need the following things:

1. A GitHub account
2. A Vercel account
3. An OpenAI API key

All of these accounts are free except the OpenAI API key requires a little bit of money to use. Demoing this notebook costs less than ten cents.

If you do not want to make an OpenAI account, you can contact me, the organizer, for a temporary key that you can use for a while.


## Step 1: Set your github email and name

If you don't have a github account, now is a great time to make one. They are free!

In [1]:
your_github_email = "alex.g.alcivar49@gmail.com"
your_github_name = "Alex Alcivar"

## Step 2: Create a private github repo

We are going to create a private repo because we are starting a business and we don't want other people to see our ideas before we are ready. Create a private repository on github, maybe called "my-create-llama-app" and copy the SSH github URL below:

In [2]:

your_github_repo = "git@github.com:mindfulcoder49/my-create-llama-app.git"

In [4]:
from google.colab import userdata
import os
os.environ['MYAPIKEY'] = userdata.get('OPENAI_API_KEY')
os.environ['MYEMAIL'] = your_github_email
os.environ['MYNAME'] = your_github_name
os.environ['MYREPO'] = your_github_repo

## Step 3: Generate SSH Keys to access your GitHub repository

You need to generate SSH Keys here on your development machine and then add the public key to your GitHub account settings. This is found by clicking the profile icon in the top right of the page on GitHub, then Settings, then SSH and GPG Keys. Then click Add, give it any name, and paste in the key generated below:

In [5]:
!ssh-keygen -t rsa -b 4096 -C $MYEMAIL

#When prompted to enter the file, just click to the right of the text and hit enter. Do this for all prompts in this notebook.

Generating public/private rsa key pair.
Enter file in which to save the key (/root/.ssh/id_rsa): 
Created directory '/root/.ssh'.
Enter passphrase (empty for no passphrase): 
Enter same passphrase again: 
Your identification has been saved in /root/.ssh/id_rsa
Your public key has been saved in /root/.ssh/id_rsa.pub
The key fingerprint is:
SHA256:PKyHHQU1lPVSFkdRovx94icjr4nYBCzIIVmTNSqfjoc alex.g.alcivar49@gmail.com
The key's randomart image is:
+---[RSA 4096]----+
|    ooo .o+o. =+*|
|   o.o . ..o = o |
|  + o     . + .  |
|   = + + .   o . |
|    = . S     o o|
|   +   = +   . ..|
|  E o o o . . + .|
|   .   . + . + + |
|        . o o..  |
+----[SHA256]-----+


In [6]:
!cat /root/.ssh/id_rsa.pub


ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAACAQC9gYgGhOEGnj+y3sQl3Vn20fns4FWXw7PPOSMposPZKhnqhOFM87ZKJYkyrv3zBpYWwTO1OGd1RPzkzq7B2IT2l64mevtpPgtjnVWJ2KtUII43+UdHY+iELwVGe9D3Fs86zfM/t4HEMqICh6NQazRoN54QOCNnp+pADitOq2ewWgxsofd+rImPpoFHifpnHSYqu0SDVNk0MnK6nOfwKDfh73c9aVtkEgwukMsq3YhEY8i8WKsIkwiWbjD5T4/0InpfzKTwejF8t3XrY9FsYHLqgVJlzWSyGADpfH27ppwqZesF6FOB9cgIE8mFNm0E/7PCyTvRxAmIu2eQ/MEksg87ovktGozxKP68KZRU+baqmHxmJ3qQ1q0QhSmAiAVKdZ/WHnmH3wzzUcsUnNNTRi75s6/D+z/MTm8pKQ5oEdy5iH2fM0rEJt3pLjXE+XkFMHmjZmkMCrrVhkWRLjHUCeMelCSzDBd2BBaTG3CXpPH8iuraBplBBgh27QoDi71w/qvax+ePimS+eYa1lGVWyb7I9jBowW+WHchydKg36xexZeE/tpc5RenQCoxJ03TaMFLDNB+trZa+3dLJX0Ssm922MdCFV7trKkdY+dmD4Zx/PDqCc5sj/+SiAL2FMrZ60T/UYNSJYZeqtFzdJJwTh+vWkvtAbkc/qgibGbvCBc1Mvw== alex.g.alcivar49@gmail.com


## Step 4: Generate the App code

In [7]:
!dpkg --configure -a
!sudo apt-get update
!sudo apt-get install -y ca-certificates curl gnupg
!sudo mkdir -p /etc/apt/keyrings
!curl -fsSL https://deb.nodesource.com/gpgkey/nodesource-repo.gpg.key | sudo gpg --dearmor -o /etc/apt/keyrings/nodesource.gpg
!NODE_MAJOR=18 && echo "deb [signed-by=/etc/apt/keyrings/nodesource.gpg] https://deb.nodesource.com/node_$NODE_MAJOR.x nodistro main" | sudo tee /etc/apt/sources.list.d/nodesource.list
!sudo apt-get update
# RUN the following commented lines to remove the previous installed nodejs (avoid conflict error)
#!sudo dpkg --remove --force-remove-reinstreq nodejs
#!sudo dpkg --remove --force-remove-reinstreq libnode-dev
#!sudo dpkg --remove --force-remove-reinstreq libnode72:amd64
!sudo apt-get install nodejs -y

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,742 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 1,975 kB in 1s (1,564 kB/s)
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state informati

In [8]:
!npx create-llama@latest my-app


#When prompted to enter input, just click to the right of the text and hit enter. Do this for all prompts in this notebook.


Need to install the following packages:
create-llama@0.0.25
Ok to proceed? (y) 
? Which template would you like to use? › - Use arrow-keys. Return to submit.
   Chat without streaming
❯  Chat with streaming
   Community template from https://github.com/run-llama/create_llama_projects
   Example using a LlamaPack✔ Which template would you like to use? › Chat with streaming
? Which framework would you like to use? › - Use arrow-keys. Return to submit.
❯  NextJS
   Express
   FastAPI (Python)✔ Which framework would you like to use? › NextJS
? Which UI would you like to use? › - Use arrow-keys. Return to submit.
❯  Just HTML
   Shadcn✔ Which UI would you like to use? › Just HTML
? Which model would you like to use? › - Use arrow-keys. Return to submit.
❯  gpt-3.5-turbo
   gpt-4-turbo-preview
   gpt-4
   gpt-4-vision-preview✔ Which model would you like to use? › gpt-3.5-turbo
? Which data source would you like to use? › - Use arrow-keys. Return to submit.
   No data, just a simple chat
❯  U

## Step 6: Push the initial app version to GitHub

We need to push the code to github with the .env, but we don't want to include our API key yet. We need to add the API later to the .env to generate our vector store here, and at that point we will also add the .env to the .gitignore

In [9]:
!git -C ./my-app init

hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/my-app/.git/


In [ ]:
# prompt: add openai_api_key to .env and add .env to .gitignore

!echo "OPENAI_API_KEY=$MYAPIKEY" >> ./my-app/.env
!echo ".env" >> ./my-app/.gitignore

In [10]:
!git -C ./my-app add .

In [11]:
!git config --global user.email $MYEMAIL

In [12]:
!git config --global user.name $MYNAME

In [13]:
!git -C ./my-app commit -m "first commit"

[master (root-commit) 8df41f8] first commit
 29 files changed, 8544 insertions(+)
 create mode 100644 .env
 create mode 100644 .eslintrc.json
 create mode 100644 .gitignore
 create mode 100644 README.md
 create mode 100644 app/api/chat/engine/constants.mjs
 create mode 100644 app/api/chat/engine/generate.mjs
 create mode 100644 app/api/chat/engine/index.ts
 create mode 100644 app/api/chat/llamaindex-stream.ts
 create mode 100644 app/api/chat/route.ts
 create mode 100644 app/components/chat-section.tsx
 create mode 100644 app/components/header.tsx
 create mode 100644 app/components/transform.ts
 create mode 100644 app/components/ui/chat/chat-avatar.tsx
 create mode 100644 app/components/ui/chat/chat-input.tsx
 create mode 100644 app/components/ui/chat/chat-item.tsx
 create mode 100644 app/components/ui/chat/chat-messages.tsx
 create mode 100644 app/components/ui/chat/index.ts
 create mode 100644 app/favicon.ico
 create mode 100644 app/globals.css
 create mode 100644 app/layout.tsx
 crea

In [14]:
!git -C ./my-app branch -M main

In [15]:
!git -C ./my-app remote add origin $MYREPO


In [16]:
!echo "Host github.com\n\tStrictHostKeyChecking no\n" >> /root/.ssh/config


In [17]:
!ssh-keyscan -t rsa github.com >> ~/.ssh/known_hosts

# github.com:22 SSH-2.0-babeld-8405f9f3


In [18]:
!git -C ./my-app push -u origin main

Enumerating objects: 40, done.
Counting objects: 100% (40/40), done.
Delta compression using up to 2 threads
Compressing objects: 100% (35/35), done.
Writing objects: 100% (40/40), 165.81 KiB | 8.73 MiB/s, done.
Total 40 (delta 0), reused 0 (delta 0), pack-reused 0
To github.com:mindfulcoder49/my-create-llama-app.git
 * [new branch]      main -> main
Branch 'main' set up to track remote branch 'main' from 'origin'.


 ## Step 7: Deploy Repository as Project on Vercel

In [ ]:
!cat my-app/.env

Copy the output from the above code and paste them into the Environment Variables section of the Vercel Project Deployment page. Then right-click on the output above and select "Clear Output" to protect your API key.

## Step 8a: Generate Vector Embeddings for RAG

You can run step 8 repeatedly after adding more documents to the data folder to add knowledge to your RAG app

In [48]:
#First we pull in case we have made changes to the interface in vscode.dev (see step 10)
!git -C my-app pull

remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 5 (delta 3), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (5/5), 1.16 KiB | 592.00 KiB/s, done.
From github.com:mindfulcoder49/my-create-llama-app
   eb269bd..82b4835  main       -> origin/main
Updating eb269bd..82b4835
Fast-forward
 app/components/header.tsx | 6 +-----
 1 file changed, 1 insertion(+), 5 deletions(-)


In [49]:
!npm -C my-app run generate


> my-app@0.1.0 generate
> node app/api/chat/engine/generate.mjs

Generating storage context...
Storage context successfully generated in 1.998s.
Finished generating storage.


## Step 8b: Push Embeddings to GitHub

In [50]:
!git -C ./my-app add .

In [51]:
!git -C ./my-app commit -m "add vector store for RAG"

[main 7c92fe0] add vector store for RAG
 4 files changed, 3 insertions(+), 3 deletions(-)
 rewrite cache/doc_store.json (90%)
 create mode 100644 data/Alex Rich-Shea Resume 2023.pdf


In [52]:
!git -C ./my-app push -u origin main

Enumerating objects: 14, done.
Counting objects: 100% (14/14), done.
Delta compression using up to 2 threads
Compressing objects: 100% (8/8), done.
Writing objects: 100% (8/8), 57.07 KiB | 2.85 MiB/s, done.
Total 8 (delta 4), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (4/4), completed with 4 local objects.
To github.com:mindfulcoder49/my-create-llama-app.git
   82b4835..7c92fe0  main -> main
Branch 'main' set up to track remote branch 'main' from 'origin'.


 ## Step 10: Use vscode.dev to work on your app code

 Google Colab is great for running the backend functions on our app to create our vector embeddings, but it doesn't have a good way to edit our codebase to change the frontend interface. For that we can use vscode! We will keep Google Colab and VsCode in sync using git version control! Hooray!